In [18]:
import pandas as pd
import numpy as np

In [19]:
u_cols = ["user_id", "age", "sex", "occupation", "zip_code"]
users = pd.read_csv(
    "./ml-100k/u.user",
    sep="|",
    names=u_cols,
    encoding="latin-1",
)
i_cols = [
    "movie_id",
    "title",
    "release date",
    "video release date",
    "IMDB URL",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
movies = pd.read_csv(
    "./ml-100k/u.item",
    sep="|",
    names=i_cols,
    encoding="latin-1",
)
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(
    "./ml-100k/u.data",
    sep="\t",
    names=r_cols,
    encoding="latin-1",
)
# timestamp 제거
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

In [20]:
# train test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [21]:
# RMSE
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full Matrix 구하기
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [22]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0325430716879735

In [23]:
# Full matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id')

# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [24]:
### Gender 기준 추천 ###
# gender별 평균을 예측치로 돌려주는 함수
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

1.043523096422976

In [ ]:
# 1.04는 전체 평균으로 계산한 1.03보다 딱히 개선되지 않은 수치.

연습문제
- 2-1 위의 설별 추천 코드를 수정해서 사용자의 직업에 따라 집단을 나누어서 예측값을 구하는 함수를 만들고 이의 정확도를 계산하는 코드를 작성
- 2-2 사용자의 성별과 직업을 동시에 고려한 집단을 나누어서 예측값을 구하는 함수를 만들고 이의 정확도를 계산하는 코드를 작성